# Анализ открытых данных ФНС РФ  

## 0. Формирование датасетов из сырых данных

ФНС предоставляет данные в формате xml. В одном наборе данных может содержаться неколько тысяч xml-файлов, данные из которых предстоит объединить в единый датасет. 

Вспомогательные скрипты:

-  [для обработки датасета по численности](https://github.com/Karmanoid/Open-Tax-Data/blob/master/parse_FNS_people.py) 

- [для обработки датасета по бухгалтерской отчетности](https://github.com/Karmanoid/Open-Tax-Data/blob/master/parse_FNS_fin_report.py)

- [для обработки датасета по уплаченным налогам](https://github.com/Karmanoid/Open-Tax-Data/blob/master/parse_FNS_taxes.py)

## 1. Данные о численности сотрудников

Вспомогательные скрипты, которые выдают аналитику по данным

In [11]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from pandas import ExcelWriter


def people_distribution (df, data='People'):
    """
    функция на вход берет датасет с численностью сотрудников (по умолчанию колонка - People)
    и выдает распределение компаний по численности сотрудниковв датасете 
    """
    zero=df[df[data]==0]
    print ('\n Компаний со штатом 0 чел.: ',zero[data].count ())

    non_zero=df[df[data]!=0]

    one_man_army=df[df[data]==1]
    print ('\n Компаний со штатом 1 чел.: ',one_man_army[data].count ())


    man_10=df[(df[data]>1)&(df[data]<=10)]
    print ('\n Компаний со штатом от 2 до 10 чел.: ', man_10[data].count ())


    man_100=df[(df[data]>10)&(df[data]<=100)]
    print ('\n Компаний со штатом от 11 до 100 чел.: ', man_100[data].count ())

    man_1000=df[(df[data]>100)&(df[data]<=1000)]
    print ('\n Компаний со штатом от 101 до 10000 чел.: ', man_1000[data].count ())

    man_10000=df[(df[data]>1000)&(df[data]<=10000)]
    print ('\n Компаний со штатом от 1001 до 10000 чел.: ', man_10000[data].count ())

    man_10000_more=df[df[data]>10001]
    print ('\n Компаний со штатом больше 10000 чел.: ', man_10000_more[data].count ())

# загружаем датасет полностью в память
datafile="./2018_people.pkl"
df=pd.read_pickle(datafile)

print ('start')
print ('Общая информация о датасете:')
print ('Всего записей в датасете:',df['People'].count (),'\n',df.describe(),'\n')

people_distribution (df)



top100=df.nlargest(100,'People') # выбираем ТОП-100 компаний по численности

#print (top100.head()) # печатаем 5 компаний из нового датасета

# собираем данные в разрезе региона: медиана численности, стандартное отклонение, мин и макс значение 
top_region=df.groupby(['region'])['region','People'].agg([np.mean, np.std, np.min, np.max])


# сохраняем данные в единый файл в формате Excel

writer = ExcelWriter ('./top_workers_2018.xlsx')

# создаем листы

top100.to_excel(writer,'Top-100') # ТОП-100 копаний по численности
top_region.to_excel(writer,'Regions') # сводная информация по регионам

writer.save()

start
Общая информация о датасете:
Всего записей в датасете: 2380191 
              People
count  2.380191e+06
mean   8.749879e+00
std    6.804749e+01
min    0.000000e+00
25%    1.000000e+00
50%    1.000000e+00
75%    4.000000e+00
max    1.627600e+04 


 Компаний со штатом 0 чел.:  323302

 Компаний со штатом 1 чел.:  991147

 Компаний со штатом от 2 до 10 чел.:  777067

 Компаний со штатом от 11 до 100 чел.:  263295

 Компаний со штатом от 101 до 10000 чел.:  23926

 Компаний со штатом от 1001 до 10000 чел.:  1442

 Компаний со штатом больше 10000 чел.:  12


In [17]:
"""
объединение данных за разные года и подготовка аналитики
""" 

import pandas as pd
import numpy as np

# грузим датасеты

datafile=("./2017_people.pkl")
df1=pd.read_pickle(datafile)
#print (df1.info())
df1=df1[['INN','People']]
df2=pd.read_pickle("./2018_people.pkl")
#print (df2.info())
df2=df2[['INN','People']]

#объединяем датасеты за 2017 и 2018, присваваем в новом датасете имена колонок
# объединяем по ИНН

print ('===== JOIN ====')
result=pd.merge (df1, df2, on='INN', how='outer', suffixes=('_2017','_2018'))

#print (result.head())
#print (result.info())

print ('Всего людей в 2017:', result.People_2017.sum())
print ('Всего людей в 2018:', result.People_2018.sum())

print ('===== АНАЛИЗ  ====')
# новые компании, которых не было в 2017, т.е. по ним отсутствуют данные в прошлом году
new=result[result['People_2017'].isnull()]

print ('Новых компаний:','\n')
#print (new.info(), new.head())
print ('Новых людей в 2018 году', new.People_2018.sum())

# показываем распределение по новым компаниям
people_distribution (new,data='People_2018')



print ('Старых компаний:', '\n')
old=result[(result['People_2017'] is not None) & (result['People_2018'].isnull())]
print ('уволено людей в 2018 году', old.People_2017.sum())

# показываем распределение по новым компаниям
people_distribution (old, data='People_2017')





print ('===== INTERSECT ====')
#выбираем данные тех компаниях, по которым есть данные за оба года

intersect=pd.merge (df1, df2, on='INN', how='inner', suffixes=('_2017','_2018'))

#print (intersect.head())
print ('Всего компаний в 2017 и 2018 году', intersect.People_2017.count())

# выводим общую численность по таким компаниям

print (' людей в 2017 году', intersect.People_2017.sum())
print ('людей в 2018 году', intersect.People_2018.sum())


===== JOIN ====
Всего людей в 2017: 20359612.0
Всего людей в 2018: 20826383.0
===== АНАЛИЗ  ====
Новых компаний: 

Новых людей в 2018 году 3224917.0

 Компаний со штатом 0 чел.:  62418

 Компаний со штатом 1 чел.:  180811

 Компаний со штатом от 2 до 10 чел.:  56125

 Компаний со штатом от 11 до 100 чел.:  8544

 Компаний со штатом от 101 до 10000 чел.:  2441

 Компаний со штатом от 1001 до 10000 чел.:  803

 Компаний со штатом больше 10000 чел.:  7
Старых компаний: 

уволено людей в 2018 году 2233012.0

 Компаний со штатом 0 чел.:  101555

 Компаний со штатом 1 чел.:  239552

 Компаний со штатом от 2 до 10 чел.:  73570

 Компаний со штатом от 11 до 100 чел.:  14458

 Компаний со штатом от 101 до 10000 чел.:  1224

 Компаний со штатом от 1001 до 10000 чел.:  140

 Компаний со штатом больше 10000 чел.:  2
===== INTERSECT ====
Всего компаний в 2017 и 2018 году 2069042
 людей в 2017 году 18126600
людей в 2018 году 17601466
